In [65]:
#from ipynb.fs.full.LocationCleaningScript import clean_location, standardize_cities
from ipynb.fs.full.cleanIndustries import cleanIndustries
# from ipynb.fs.full.CurrencyConverter import *
#from ipynb.fs.full.NormalizeWorkExperience import NormalizeExperience
import pandas as pd
import urllib.request
import json
# from datascience import *
def RemoveSpacesFromTitles(df):
    
    df.columns = [c.replace(' ', '_') for c in df.columns]
    return df
def NormalizeExperience(filename):
    df = pd.read_csv(filename)
    df = df.replace(to_replace = ['5-7 years'], value = '5 - 7 years')
    df = df.replace(to_replace = ['1 year or less'], value = '0 - 1 years')
    return df

In [66]:
class currencyConverter:
    rates = {}
    def __init__(self, url):
        data = urllib.request.urlopen(url).read().decode('utf-8')
        f = json.loads(data)
        self.rates = f["rates"]
        
        
    def convert(self, amount, f, t):
        output = 0
        if f != 'USD':
            output = amount / self.rates[f] * self.rates['USD']
            return output
        return amount
        
def convertCurrency(df):
    for i in range(len(df['Currency'])):
        f = df['Currency'][i]
        salary = df['Annual_Salary'][i].replace(',', '')
        otherComp = df['Other_Compensation'][i]
        if not pd.isna(df['Other_Compensation'][i]):
            salary = float(salary) + float(otherComp)
        if f == 'Other':
            otherCur = df['Other'][i]
            if otherCur == 'Peso Argentino' or otherCur == 'Argentinian peso (ARS)' or otherCur == 'Argentine Peso':
                otherCur = 'ARS'
            if otherCur == 'Indian rupees' or otherCur == 'INR (Indian Rupee)':
                otherCur = 'INR'
            if otherCur == 'BRL (R$)' or otherCur == 'BR$':
                otherCur = 'BRL'
            if otherCur == 'Dkk' or otherCur == 'Danish Kroner' or otherCur == 'dkk':
                otherCur = 'DKK'
            if otherCur == 'Mexican pesos' or otherCur == 'Mexican Pesos':
                otherCur = 'MXN'
            if otherCur == 'Bdt':
                otherCur = 'BDT'
            if otherCur == 'American Dollars':
                otherCur = 'USD'
            if otherCur == 'Php' or otherCur == 'Philippine Peso' or otherCur == 'Philippine peso (PHP)' or otherCur == 'PhP (Philippine Peso)' or otherCur == 'Philippine Pesos':
                otherCur = 'PHP'
            if otherCur == 'PLN (Polish zloty)' or otherCur == 'Polish Złoty':
                otherCur = 'PLN'
            if otherCur == 'czech crowns' or otherCur == 'Czk':
                otherCur = 'CZK'
            if otherCur == 'Norwegian kroner (NOK)' or otherCur == 'Nok':
                otherCur = 'NOK'
            if otherCur == 'ILS/NIS' or otherCur == 'NIS (new Israeli shekel)' or otherCur == 'ILS (Shekel)' or otherCur == 'Israeli Shekels' or otherCur == 'Ils':
                otherCur = 'ILS'
            if otherCur == 'US Dollar':
                otherCur = 'USD'
            if otherCur == 'RMB (chinese yuan)' or otherCur == 'China RMB' or otherCur == 'RM':
                otherCur = 'CNY'
            if otherCur == 'Taiwanese dollars' or otherCur == 'NTD':
                otherCur = 'TWD'
            if otherCur == 'KRW (Korean Won)' or otherCur == 'Korean Won ':
                otherCur = 'KRW'
            if otherCur == 'Sgd':
                otherCur = 'SGD'
            if otherCur == 'AUD Australian' or otherCur == 'Australian Dollars ':
                otherCur = 'AUD'
            if otherCur == 'Euro':
                otherCur = 'EUR'
            if otherCur == 'THAI  BAHT':
                otherCur = 'THB'
            if otherCur == 'IDR ': 
                otherCur = 'IDR'
            if otherCur in c.rates:
                df['Annual_Salary'][i] = "{:.2f}".format(c.convert(float(salary), otherCur, 'USD'))
        else:
            if f == 'AUD/NZD':
                f = 'AUD'
            if df['Other'][i] == 'COP':
                df['Annual_Salary'][i] = "{:.2f}".format(c.convert(float(salary), 'COP', 'USD'))
            df['Annual_Salary'][i] = "{:.2f}".format(c.convert(float(salary), f, 'USD'))
    return df

c = currencyConverter("http://data.fixer.io/api/latest?access_key=238f1925907e0f01a7fa1dc2b4042967")

In [67]:
def cleanGender(filename):
    df = pd.read_csv(filename, index=)
    for i in range(len(df['Gender'])):
        if df['Gender'][i] == "Prefer not to answer" or pd.isna(df['Gender'][i]):
            df['Gender'][i] = 'Other or prefer not to answer'
    return df

In [68]:
def cleanRaces(df):
    for i in range(len(df['Race'])):
        if pd.isna(df['Race'][i]):
            df['Race'][i] = 'Another option not listed here or prefer not to answer'
        if 'Hispanic, Latino, or Spanish origin' in df['Race'][i]:
            df['Race'][i] = df['Race'][i].replace('Hispanic, Latino, ', 'Hispanic Latino ')
    return df

In [55]:
table = NormalizeExperience('../Data/2021_Results.csv')

table = RemoveSpacesFromTitles(table)

table = convertCurrency(table)
table = cleanGender(table)
table = cleanRaces(table)
table = cleanIndustries(table)
#table
table.to_csv("../Data/FinalData2021.csv")

<ipython-input-2-1a58998911e8>:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Annual_Salary'][i] = "{:.2f}".format(c.convert(float(salary), f, 'USD'))
<ipython-input-2-1a58998911e8>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Annual_Salary'][i] = "{:.2f}".format(c.convert(float(salary), otherCur, 'USD'))
<ipython-input-2-1a58998911e8>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Annual_Salary'][i] = "{:.2f}".f

In [73]:
table = cleanGender('../Data/CleanedData2021.csv')
table = cleanRaces(table)
table.to_csv("../Data/FinalData2021.csv")

<ipython-input-67-959911b1d493>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Gender'][i] = 'Other or prefer not to answer'
<ipython-input-68-8945de18628f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Race'][i] = df['Race'][i].replace('Hispanic, Latino, ', 'Hispanic Latino ')
<ipython-input-68-8945de18628f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Race'][i] = 'Another option not listed here or prefer not to a